In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import joblib
# Load the dataset
dataset1 = pd.read_csv('E://PROJECTMINIS//DATASETS//multilingual//Dataset//Training//MultiLanguageTrainDataset.csv')
dataset2= pd.read_csv('E://PROJECTMINIS//DATASETS//multilingual//french//fr_hf.csv')
dataset2.rename(columns={'labels': 'label'}, inplace=True)

dataset2['language'] = 4


dataset2 = dataset2.drop(columns=['target', 'source','dataset','nb_annotators','tweet_id'])
dataset1 = dataset1.drop(columns=['Unnamed: 0'])
# Filter English data
dataset1 = dataset1[dataset1['language'] == 4]

french_data = pd.concat([dataset1, dataset2], ignore_index=True)
# Split the data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    french_data['text'].astype(str), 
    french_data['label'], 
    test_size=0.2, 
    random_state=42
)

# Tokenization and preprocessing
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
val_data = pad_sequences(val_sequences, maxlen=max_sequence_length)



In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import joblib

# Load datasets
dataset1 = pd.read_csv('E://PROJECTMINIS//DATASETS//multilingual//Dataset//Training//MultiLanguageTrainDataset.csv')
dataset2 = pd.read_csv('E://PROJECTMINIS//DATASETS//multilingual//french//fr_hf.csv')
dataset2.rename(columns={'labels': 'label'}, inplace=True)
dataset2['language'] = 4
dataset2 = dataset2.drop(columns=['target', 'source', 'dataset', 'nb_annotators', 'tweet_id'])
dataset1 = dataset1.drop(columns=['Unnamed: 0'])

# Filter English data
dataset1 = dataset1[dataset1['language'] == 4]

# Combine datasets
french_data = pd.concat([dataset1, dataset2], ignore_index=True)

# Split into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(
    french_data['text'].astype(str),
    french_data['label'],
    test_size=0.2,
    random_state=42
)

# Tokenization and preprocessing
max_words = 10000
max_sequence_length = 100

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

train_sequences = tokenizer.texts_to_sequences(train_texts)
val_sequences = tokenizer.texts_to_sequences(val_texts)

train_data = pad_sequences(train_sequences, maxlen=max_sequence_length)
val_data = pad_sequences(val_sequences, maxlen=max_sequence_length)

# Model definition
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_sequence_length),
    Conv1D(filters=64, kernel_size=5, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # For binary classification
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(train_data, train_labels, validation_data=(val_data, val_labels), epochs=10, batch_size=64, callbacks=[early_stopping])

# Load the test data
test = pd.read_csv('E://PROJECTMINIS//DATASETS//multilingual//Dataset//Testing//Test1//French_test.csv')
test_texts = test['text'].astype(str)
test_labels = test['label'].astype(int)

# Preprocess the test data
test_sequences = tokenizer.texts_to_sequences(test_texts)
test_data = pad_sequences(test_sequences, maxlen=max_sequence_length)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_data, test_labels)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate predictions
predictions = model.predict(test_data)
predictions_binary = (predictions > 0.5).astype(int)

# Classification report
print(classification_report(test_labels, predictions_binary))


C:\Users\Akshat\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - accuracy: 0.6663 - loss: 0.6030 - val_accuracy: 0.7548 - val_loss: 0.4681
Epoch 2/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.7993 - loss: 0.4151 - val_accuracy: 0.7538 - val_loss: 0.4869
Epoch 3/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 12s 35ms/step - accuracy: 0.8756 - loss: 0.2842 - val_accuracy: 0.7536 - val_loss: 0.5343
Epoch 4/10
350/350 ━━━━━━━━━━━━━━━━━━━━ 13s 36ms/step - accuracy: 0.9167 - loss: 0.1967 - val_accuracy: 0.7500 - val_loss: 0.5957
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7728 - loss: 0.4739
Test Loss: 0.4702
Test Accuracy: 0.7758
35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step
              precision    recall  f1-score   support

           0       0.74      0.82      0.78       532
           1       0.81      0.73      0.77       565

    accuracy                           0.78      1097
   macro avg       0.78      0.78      0.78      1097
weighted avg       0.78      0.78      0.78      1097


In [26]:
def predict_input_with_percentage(text, tokenizer, model, max_sequence_length=100):
    
    sequences = tokenizer.texts_to_sequences([text])  
    padded_sequence = pad_sequences(sequences, maxlen=max_sequence_length)  
    
    
    prediction = model.predict(padded_sequence)  
    
    
    prediction_percentage = prediction[0][0] * 100  
    
   
    predicted_class = (prediction > 0.5).astype(int)  
    
    return predicted_class[0][0], prediction_percentage


# Example input texts
input_texts = [
    "Les Noirs sont des sauvages qui devraient être renvoyés en Afrique à coups de fouet",
    "Les Arabes ne s’intégreront jamais, ils veulent imposer la charia partout",
    "J’aime découvrir les cultures du monde, chaque pays a quelque chose d’unique à offrir.",
    "la souillion"
]

# Test each input and display the class with percentage
for text in input_texts:
    predicted_class, prediction_percentage = predict_input_with_percentage(text, tokenizer, model)
    
    # Print the result
    if predicted_class == 1:
        print(f"Text: '{text}' - Predicted: Hate Speech (Class 1) - Probability: {prediction_percentage:.2f}%")
    else:
        print(f"Text: '{text}' - Predicted: Normal Speech (Class 0) - Probability: {prediction_percentage:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Text: 'Les Noirs sont des sauvages qui devraient être renvoyés en Afrique à coups de fouet' - Predicted: Hate Speech (Class 1) - Probability: 74.73%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Text: 'Les Arabes ne s’intégreront jamais, ils veulent imposer la charia partout' - Predicted: Hate Speech (Class 1) - Probability: 56.81%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Text: 'J’aime découvrir les cultures du monde, chaque pays a quelque chose d’unique à offrir.' - Predicted: Normal Speech (Class 0) - Probability: 7.18%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
Text: 'la souillion' - Predicted: Normal Speech (Class 0) - Probability: 11.11%


In [18]:
joblib.dump(model, 'french_model.pkl')

['french_model.pkl']

In [20]:
import pickle
from tensorflow.keras.preprocessing.text import Tokenizer



# Save the tokenizer as a .pkl file
with open('tokenizer_french.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

print("Tokenizer saved to tokenizer.pkl")

Tokenizer saved to tokenizer.pkl
